In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import scipy.integrate as spint
import scipy.interpolate as spinter
from multiprocessing import Process, shared_memory, cpu_count
import lense_thirring_tools as ltt
#from manim_tools import ParametricCurve

In [2]:
M = 1
omega = 10.0
R = 1
S = 2.0/5.0 * M * R**2 * omega
ltt.set_params_lense_thirring(mass=M, omega=omega, radius=R)

In [ ]:
num_threads = 6
Nps = 1000
cputmax = 10
tmax = 10.0
timaxs = 10000
startRegion = np.array([[2,-0.5,-0.5],[3,0.5,0.5]])
boundary = np.array([[-0.1,-3,-3,-3,-np.Inf,-np.Inf,-np.Inf,-np.Inf],[np.Inf,3,3,3,np.Inf,np.Inf,np.Inf,np.Inf]])
startPoints = np.random.random((Nps,3))*(startRegion[1]-startRegion[0]) + startRegion[0]
print(f"using startPoints: {startPoints.shape}")

# create array with np data and init ParametricCurve from that afterwards
pnt_ev = np.empty((Nps,timaxs,4), dtype=np.float64)

def simulate_lines(pis, shmn):
    global Nps, cputmax, startPoints
    existing_shm = shared_memory.SharedMemory(name=shmn)
    pnt_ev = np.ndarray((Nps,timaxs,4), dtype=np.float64, buffer=existing_shm.buf)
    for i,sp in enumerate(startPoints[pis]):
        pi = pis[i]
        z0 = np.array([0.0,sp[0],sp[1],sp[2],1.0,0,0,0])
        ts, ps, status = ltt.get_geodesic2(ltt.acc_lense_thirring,z0,1.0,cputmax=cputmax,boundary=boundary,recurring_tol=1e-14,tmax=tmax)
        if status != 'finished' and status != 'running':
            print(status)
        if len(ts) > timaxs-1:
            print('array too large')
            ts = ts[:timaxs-1]
            ps = ps[:timaxs-1]
        pnt_ev[pi,-1,0] = len(ts)
        pnt_ev[pi,:len(ts),0] = ts
        pnt_ev[pi,:len(ts),1:] = ps[:,1:4]

shm = shared_memory.SharedMemory(create=True, size=pnt_ev.nbytes)
pnt_ev = np.ndarray((Nps,timaxs,4), dtype=np.float64, buffer=shm.buf)

ts = []
for thread_i in range(num_threads):
    pis = np.arange(thread_i, len(startPoints), num_threads)
    ts.append(Process(target=simulate_lines, args=[pis, shm.name]))
    ts[-1].start()

for t in ts:
    t.join()

pcs = [ltt.ParametricCurve(ev[:int(ev[-1,0]),0],ev[:int(ev[-1,0]),1:]) for ev in pnt_ev]
tmax = np.amax([pc.tmax for pc in pcs])

using startPoints: (1000, 3)


In [4]:
%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

fig.subplots_adjust(bottom=0.25)

ax.set_aspect('equal')
lim = 3.5
ax.set(xlim=[-lim,lim], ylim=[-lim,lim], zlim3d=[-lim,lim], xlabel=r'x', ylabel=r'y')
ax.view_init(elev=10., azim=(-100))

pnts = np.array([pc.x_of_l(pc.l_of_t(0.0)) for pc in pcs])
scat = ax.scatter(pnts[:,0],pnts[:,1],pnts[:,2],c=[0,0,0,0.2],depthshade=False)

tau_slider_ax  = fig.add_axes([0.25, 0.15, 0.65, 0.03])
tau_slider = Slider(tau_slider_ax, 'tau idx', 0.0, tmax, valstep=tmax/100, valinit=0.0)

angle_slider_ax = fig.add_axes([0.25, 0.1, 0.65, 0.03])
angle_slider = Slider(angle_slider_ax, 'angle', 0, 360, valinit=0)

def tau_on_changed(val):
    tau = tau_slider.val
    fig.suptitle(rf'$\tau={tau:0.2f}$')
    pnts = np.array([pc.x_of_l(pc.l_of_t(tau)) for pc in pcs])
    scat._offsets3d = (pnts[:,0],pnts[:,1],pnts[:,2])
    fig.canvas.draw_idle()
def angle_on_changed(val):
    angle = angle_slider.val
    ax.view_init(elev=10., azim=(-angle-100))
    fig.canvas.draw_idle()
tau_slider.on_changed(tau_on_changed)
angle_slider.on_changed(angle_on_changed)

plt.show()

/usr/local/lib/python3.10/dist-packages/scipy-1.13.0-py3.10-linux-x86_64.egg/scipy/interpolate/_interpolate.py:712: RuntimeWarning: invalid value encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/tmp/ipykernel_77504/2445353871.py:14: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  scat = ax.scatter(pnts[:,0],pnts[:,1],pnts[:,2],c=[0,0,0,0.2],depthshade=False)
